# Improved LSTM Model and Portfolio Optimization

This notebook implements an enhanced LSTM model with better architecture and robust portfolio optimization using modern portfolio theory with ESG constraints.

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import TimeSeriesSplit
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.optimize import minimize
import yfinance as yf
%matplotlib inline

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
merged_df = pd.read_csv("data/processed/MultiIndex_stock_data.csv", index_col=0)
merged_df.index = pd.to_datetime(merged_df.index)
esg_scores = pd.read_csv("data/raw_data/esg_scores.csv", index_col=0)

## 1. Improved Data Preprocessing

Implementing robust data preprocessing with better feature scaling and cross-validation.

In [ ]:
def prepare_sequence_data(data, lookback=60, forecast_horizon=1):
    """
    Prepare sequence data with proper padding and scaling
    """
    # Standard scaling for better gradient flow
    scaler = StandardScaler()
    scaled_data = scaler.fit_transform(data)
    
    # Create sequences with proper padding
    X, y = [], []
    for i in range(len(scaled_data) - lookback - forecast_horizon + 1):
        X.append(scaled_data[i:(i + lookback)])
        y.append(scaled_data[i + lookback:i + lookback + forecast_horizon, 0])
    
    return np.array(X), np.array(y), scaler

def create_time_series_cv(n_splits=5):
    """
    Create time series cross-validation splits
    """
    return TimeSeriesSplit(n_splits=n_splits, test_size=int(len(merged_df) * 0.2))

# Prepare features for each stock
stock_features = {}
for ticker in merged_df.columns.levels[0]:
    stock_data = merged_df[ticker].copy()
    
    # Additional features
    features = pd.DataFrame({
        'price': stock_data['Adj Close'],
        'volume': stock_data['Volume'],
        'daily_return': stock_data['DailyRet'],
        'momentum_20d': stock_data['20DayRet'],
        'volatility_20d': stock_data['20DayVol'],
        'z_score_ret': stock_data['Z20DayRet'],
        'z_score_vol': stock_data['Z20DayVol'],
        'esg_score': pd.Series(esg_scores.loc[ticker, 'esg_score'], index=stock_data.index)
    })
    
    # Handle missing data
    features = features.fillna(method='ffill').fillna(method='bfill')
    stock_features[ticker] = features

print("Features prepared for all stocks")

## 2. Enhanced LSTM Model Architecture

Implementing a bidirectional LSTM with attention and batch normalization for better performance.

In [ ]:
def create_improved_lstm_model(input_shape, lstm_units=50):
    """
    Create an improved LSTM model with bidirectional layers and attention
    """
    inputs = tf.keras.Input(shape=input_shape)
    
    # Batch normalization on input
    x = layers.BatchNormalization()(inputs)
    
    # Bidirectional LSTM layers
    lstm1 = layers.Bidirectional(layers.LSTM(lstm_units, return_sequences=True))(x)
    lstm1 = layers.BatchNormalization()(lstm1)
    lstm1 = layers.Dropout(0.2)(lstm1)
    
    lstm2 = layers.Bidirectional(layers.LSTM(lstm_units, return_sequences=True))(lstm1)
    lstm2 = layers.BatchNormalization()(lstm2)
    
    # Attention mechanism
    attention = layers.Dense(1, activation='tanh')(lstm2)
    attention = layers.Flatten()(attention)
    attention = layers.Activation('softmax')(attention)
    attention = layers.RepeatVector(lstm_units * 2)(attention)
    attention = layers.Permute([2, 1])(attention)
    
    # Merge attention with LSTM output
    merged = layers.Multiply()([lstm2, attention])
    merged = layers.Lambda(lambda x: tf.reduce_sum(x, axis=1))(merged)
    
    # Dense layers with residual connections
    dense1 = layers.Dense(32, activation='relu')(merged)
    dense1 = layers.BatchNormalization()(dense1)
    dense1 = layers.Dropout(0.2)(dense1)
    
    dense2 = layers.Dense(16, activation='relu')(dense1)
    dense2 = layers.BatchNormalization()(dense2)
    
    outputs = layers.Dense(1, activation='linear')(dense2)
    
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    
    # Compile with Huber loss for robustness to outliers
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
    model.compile(optimizer=optimizer, 
                 loss=tf.keras.losses.Huber(),
                 metrics=['mae', 'mse'])
    
    return model

# Train models with cross-validation
cv = create_time_series_cv()
stock_predictions = {}
stock_models = {}

for ticker in stock_features.keys():
    print(f"\nTraining model for {ticker}")
    features = stock_features[ticker].values
    
    # Prepare sequences
    X, y, scaler = prepare_sequence_data(features)
    
    # Initialize lists for cross-validation results
    val_predictions = []
    val_indices = []
    
    # Cross-validation training
    for fold, (train_idx, val_idx) in enumerate(cv.split(X)):
        print(f"Fold {fold + 1}/5")
        
        X_train, X_val = X[train_idx], X[val_idx]
        y_train, y_val = y[train_idx], y[val_idx]
        
        # Create and train model
        model = create_improved_lstm_model(input_shape=(X.shape[1], X.shape[2]))
        
        # Callbacks
        early_stopping = tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=10,
            restore_best_weights=True
        )
        
        # Train the model
        history = model.fit(
            X_train, y_train,
            epochs=50,
            batch_size=32,
            validation_data=(X_val, y_val),
            callbacks=[early_stopping],
            verbose=0
        )
        
        # Store predictions
        val_predictions.extend(model.predict(X_val).flatten())
        val_indices.extend(val_idx)
    
    # Store final model and predictions
    stock_models[ticker] = model
    stock_predictions[ticker] = pd.Series(
        scaler.inverse_transform(np.array(val_predictions).reshape(-1, 1)).flatten(),
        index=merged_df.index[60:][val_indices]
    )
    
    # Print final metrics
    final_mse = np.mean((stock_predictions[ticker] - features[60:, 0][val_indices]) ** 2)
    print(f"{ticker} - Final MSE: {final_mse:.6f}")

# Save predictions
predictions_df = pd.DataFrame(stock_predictions)
predictions_df.to_csv("data/processed/lstm_predictions.csv")

## 3. Portfolio Optimization Setup

Prepare the predicted returns and risk metrics for portfolio optimization with ESG constraints.

In [ ]:
# Calculate expected returns and covariance matrix
returns = predictions_df.pct_change().dropna()
exp_returns = returns.mean()
cov_matrix = returns.cov()

# Add ESG constraints
esg_threshold = 0.6  # Minimum ESG score threshold
min_esg_weight = 0.4  # Minimum portfolio weight for high ESG stocks

# Identify high ESG stocks
high_esg_stocks = esg_scores[esg_scores['esg_score'] >= esg_threshold].index

def portfolio_stats(weights):
    """
    Calculate portfolio statistics
    """
    portfolio_return = np.sum(exp_returns * weights)
    portfolio_risk = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
    sharpe_ratio = portfolio_return / portfolio_risk
    
    # Calculate ESG score
    portfolio_esg = np.sum(esg_scores['esg_score'] * weights)
    
    return portfolio_return, portfolio_risk, sharpe_ratio, portfolio_esg

def optimize_portfolio(target_return=None):
    """
    Optimize portfolio weights with ESG constraints
    """
    n_assets = len(exp_returns)
    
    # Initial weights
    weights = np.array([1/n_assets] * n_assets)
    
    # Constraints
    constraints = [
        {'type': 'eq', 'fun': lambda x: np.sum(x) - 1},  # Sum of weights = 1
        {'type': 'ineq', 'fun': lambda x: np.sum(x[esg_scores.index.isin(high_esg_stocks)]) - min_esg_weight}  # ESG constraint
    ]
    
    if target_return is not None:
        constraints.append({
            'type': 'eq',
            'fun': lambda x: np.sum(exp_returns * x) - target_return
        })
    
    # Bounds for individual stock weights (0-20%)
    bounds = tuple((0, 0.2) for asset in range(n_assets))
    
    # Objective function (minimize volatility)
    def objective(weights):
        return np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
    
    # Optimize
    result = minimize(objective, weights, method='SLSQP',
                     bounds=bounds, constraints=constraints)
    
    return result.x if result.success else None

# Calculate efficient frontier points
target_returns = np.linspace(exp_returns.min(), exp_returns.max(), 50)
efficient_portfolios = []

for target in target_returns:
    weights = optimize_portfolio(target)
    if weights is not None:
        stats = portfolio_stats(weights)
        efficient_portfolios.append({
            'Return': stats[0],
            'Risk': stats[1],
            'Sharpe': stats[2],
            'ESG Score': stats[3],
            'Weights': weights
        })

## 4. Portfolio Analysis and Visualization

Analyze the optimized portfolios and create visualizations of the efficient frontier and portfolio weights.

In [ ]:
# Convert efficient portfolios to DataFrame
ef_df = pd.DataFrame([{
    'Return': p['Return'],
    'Risk': p['Risk'],
    'Sharpe': p['Sharpe'],
    'ESG Score': p['ESG Score']
} for p in efficient_portfolios])

# Plot efficient frontier
plt.figure(figsize=(12, 8))
plt.scatter(ef_df['Risk'], ef_df['Return'], c=ef_df['ESG Score'], 
           cmap='viridis', s=50)
plt.colorbar(label='ESG Score')
plt.xlabel('Portfolio Risk (Volatility)')
plt.ylabel('Expected Return')
plt.title('Efficient Frontier with ESG Scores')
plt.show()

# Find optimal portfolio (highest Sharpe ratio)
optimal_idx = ef_df['Sharpe'].idxmax()
optimal_portfolio = efficient_portfolios[optimal_idx]

# Plot optimal portfolio weights
optimal_weights = pd.Series(
    optimal_portfolio['Weights'], 
    index=exp_returns.index
).sort_values(ascending=True)

plt.figure(figsize=(15, 8))
optimal_weights.plot(kind='bar')
plt.title('Optimal Portfolio Weights')
plt.xlabel('Stocks')
plt.ylabel('Weight')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Print portfolio statistics
print("\nOptimal Portfolio Statistics:")
print(f"Expected Return: {optimal_portfolio['Return']:.4f}")
print(f"Risk: {optimal_portfolio['Risk']:.4f}")
print(f"Sharpe Ratio: {optimal_portfolio['Sharpe']:.4f}")
print(f"ESG Score: {optimal_portfolio['ESG Score']:.4f}")

# Calculate maximum drawdown
def calculate_max_drawdown(returns):
    cumulative = (1 + returns).cumprod()
    rolling_max = cumulative.expanding().max()
    drawdowns = cumulative / rolling_max - 1
    return drawdowns.min()

# Calculate portfolio returns
portfolio_returns = returns.dot(optimal_portfolio['Weights'])
max_drawdown = calculate_max_drawdown(portfolio_returns)
print(f"Maximum Drawdown: {max_drawdown:.4f}")

# Save optimal portfolio
optimal_portfolio_df = pd.DataFrame({
    'Weight': optimal_weights,
    'ESG Score': esg_scores['esg_score']
})
optimal_portfolio_df.to_csv('data/processed/optimal_portfolio.csv')